In [1]:
import os
import uuid
import multiprocessing
from functools import reduce

import cv2

import numpy as np

import torch
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset, sampler

from matplotlib import pyplot as plt

from pdb import set_trace

from ddh import *
from dataset import *
from logger import *

%load_ext autoreload
%autoreload 2

In [2]:
# reset the data

undo_create_set("val")
undo_create_set("test")
create_set("val")
create_set("test")

In [3]:
# switch to the appropriate device

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [4]:
# hyperparameters

num_epochs = 1000

optimizer_params = {
    "lr": 1e-5,
    "weight_decay": 0.0
}
custom_params = {
    "beta": 0.0 # regularization to guide bits to either 1 or -1
}

# how many bits to map the images to with the network
hash_dim = 48

In [5]:
# add data transformation here

transform = T.Compose([
    T.ToPILImage(),
    T.Resize((32, 32)),
    T.ToTensor()
])

In [6]:
# build the dataset 

data_train = FaceScrubDataset(type="hash_label",
                              mode="train",
                              transform=transform,
                              hash_dim=hash_dim)
data_val = FaceScrubDataset(type="hash_label",
                            mode="val",
                            transform=transform,
                            hash_dim=hash_dim)
data_test = FaceScrubDataset(type="hash_label",
                             mode="test",
                             transform=transform,
                             hash_dim=hash_dim)

In [7]:
# setting up the data loader

batch_size = {
    "train": 256,
    "val": 256,
    "test": 256
}

loader_params = {
    "shuffle": True,
    "num_workers": multiprocessing.cpu_count() - 2
}

loader_train = DataLoader(data_train,
                          batch_size=batch_size["train"],
                          **loader_params)
loader_val = DataLoader(data_val,
                          batch_size=batch_size["val"],
                          **loader_params)
loader_test = DataLoader(data_test,
                          batch_size=batch_size["test"],
                          **loader_params)

In [8]:
# components

model = DiscriminativeDeepHashing(hash_dim=hash_dim)
model.to(device=device)
optimizer = optim.Adam(model.parameters(), **optimizer_params)

run_id = uuid.uuid4().hex.upper()[0:6]
checkpoint_path = os.getcwd() + "/models/{}_best_weights.pt".format(run_id)

In [9]:
# training code

train_losses = []
train_acc = []
val_acc = []

with Logger(write_to_file=False) as logger:
    logger.write("Starting run for {}".format(run_id))
    logger.write("====== START ======")

    for epoch in range(num_epochs):
        epoch_train_losses = []
        epoch_train_data_correct = []
        epoch_train_data_total = []
        epoch_train_bits_correct = []
        epoch_train_bits_total = []

        for num_iter, (X, y) in enumerate(loader_train):
            # set model to train mode
            model.train()
            # forward pass
            batch = X.to(device).float()
            label = y.to(device).float()
            outputs = model(X)
            # calculating loss
            loss = F.binary_cross_entropy_with_logits(outputs, label)
            # loss to encourage code to be closer to -1 and 1
            loss += custom_params["beta"] * \
                        torch.abs((torch.abs(outputs)-1).sum()) / len(X)
            # back propagate
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # calculate accuracy over training data
            y = y.byte()
            output_hash = outputs.data > 0
            # a classification is considered correct if the output matches the target bits exactly
            num_correct = ((output_hash == y).sum(1) == hash_dim).sum().item()
            # the number of bits that are correct
            num_bits_correct = (output_hash == y).sum().item()
            num_bits_total = reduce(lambda x, y: x*y, output_hash.shape)
            # calculate the percentage
            train_data_acc = num_correct * 100.0 / len(X)
            train_bits_acc = num_bits_correct * 100.0 / num_bits_total
            # save statistics
            epoch_train_losses.append(loss.item())
            epoch_train_data_correct.append(num_correct)
            epoch_train_data_total.append(len(X))
            epoch_train_bits_correct.append(num_bits_correct)
            epoch_train_bits_total.append(num_bits_total)

            if num_iter % 200 == 0:
                print(
                    "TRAIN epoch {} ".format(epoch) +
                    "iter {}: ".format(num_iter) +
                    "loss - {:.10f}, ".format(loss.item()) +
                    "correct/total - {}/{} ({:.2f}%), "
                        .format(num_correct, len(X), train_data_acc) +
                    "bits/total - {}/{} ({:.2f}%). "
                        .format(num_bits_correct, num_bits_total, train_bits_acc)
                )
        
        epoch_val_data_correct = []
        epoch_val_bits_correct = []
        epoch_val_data_total = []
        epoch_val_bits_total = []
        
        for num_iter, (X, y) in enumerate(loader_val):
            model.eval()
            batch = X.to(device).float()
            label = y.to(device).float()
            outputs = model(X)
            # calculate accuracy over validation data
            y = y.byte()
            output_hash = outputs.data > 0
            # number of data points correct
            num_correct = ((output_hash == y).sum(1) == hash_dim).sum().item()
            # number of bits correct
            num_bits_correct = (output_hash == y).sum().item()
            num_bits_total = reduce(lambda x, y: x*y, output_hash.shape)
            # append to history
            epoch_val_data_correct.append(num_correct)
            epoch_val_data_total.append(len(X))
            epoch_val_bits_correct.append(num_bits_correct)
            epoch_val_bits_total.append(num_bits_total)
            
            
            # calculate the loss and accuracy over the whole dataset by averaging them
            final_train_loss = reduce(lambda x,y: x+y, epoch_train_losses) / len(epoch_train_losses)
            final_train_data_correct = reduce(lambda x,y: x+y, epoch_train_data_correct) / len(epoch_train_data_correct)
            final_train_data_total = reduce(lambda x,y: x+y, epoch_train_data_total) / len(epoch_train_data_total)
            final_train_bits_correct = reduce(lambda x,y: x+y, epoch_train_bits_correct) / len(epoch_train_bits_correct)
            final_train_bits_total = reduce(lambda x,y: x+y, epoch_train_bits_total) / len(epoch_train_bits_total)
        
            final_val_data_correct = reduce(lambda x,y: x+y, epoch_val_data_correct) / len(epoch_val_data_correct)
            final_val_data_total = reduce(lambda x,y: x+y, epoch_val_data_total) / len(epoch_val_data_total)
            final_val_bits_correct = reduce(lambda x,y: x+y, epoch_val_bits_correct) / len(epoch_val_bits_correct)
            final_val_bits_total = reduce(lambda x,y: x+y, epoch_val_bits_total) / len(epoch_val_bits_total)
        
        print(
            "TRAIN epoch {} completed! Avg train loss - {:.6f}".format(epoch, final_train_loss) +
            "\n\tTRAIN data correct/total - {}/{} ({:.2f}%), bits correct/total - {}/{} ({:.2f}%)"
                .format(final_train_data_correct, final_train_data_total,
                        round(final_train_data_correct * 100.0 / final_train_data_total),
                        final_train_bits_correct, final_train_bits_total, 
                        round(final_train_bits_correct * 100.0 /  final_train_bits_total)) +
            "\n\tVAL   data correct/total - {}/{} ({:.2f}%), bits correct/total - {}/{} ({:.2f}%)"
                .format(final_val_data_correct, final_val_data_total, 
                        round(final_val_data_correct * 100.0 / final_val_data_total),
                        final_val_bits_correct, final_val_bits_total,
                        round(final_val_bits_correct * 100.0 / final_val_bits_total))
        )
            
    logger.write("====== END ======")
    logger.write("Completed run for {}".format(run_id))

Starting run for 2CC8CA
====== START ======
TRAIN epoch 0 iter 0: loss - 0.7089721560, correct/total - 0/10 (0.00%), bits/total - 203/480 (42.29%). 
TRAIN epoch 0 completed! Avg train loss - 0.708972
	TRAIN data correct/total - 0.0/10.0 (0.00%), bits correct/total - 203.0/480.0 (42.00%)
	VAL   data correct/total - 0.0/10.0 (0.00%), bits correct/total - 220.0/480.0 (46.00%)
TRAIN epoch 1 iter 0: loss - 0.7084426284, correct/total - 0/10 (0.00%), bits/total - 203/480 (42.29%). 
TRAIN epoch 1 completed! Avg train loss - 0.708443
	TRAIN data correct/total - 0.0/10.0 (0.00%), bits correct/total - 203.0/480.0 (42.00%)
	VAL   data correct/total - 0.0/10.0 (0.00%), bits correct/total - 220.0/480.0 (46.00%)
TRAIN epoch 2 iter 0: loss - 0.7079147696, correct/total - 0/10 (0.00%), bits/total - 206/480 (42.92%). 
TRAIN epoch 2 completed! Avg train loss - 0.707915
	TRAIN data correct/total - 0.0/10.0 (0.00%), bits correct/total - 206.0/480.0 (43.00%)
	VAL   data correct/total - 0.0/10.0 (0.00%), bi

TRAIN epoch 25 iter 0: loss - 0.6964861751, correct/total - 0/10 (0.00%), bits/total - 235/480 (48.96%). 
TRAIN epoch 25 completed! Avg train loss - 0.696486
	TRAIN data correct/total - 0.0/10.0 (0.00%), bits correct/total - 235.0/480.0 (49.00%)
	VAL   data correct/total - 0.0/10.0 (0.00%), bits correct/total - 248.0/480.0 (52.00%)
TRAIN epoch 26 iter 0: loss - 0.6960212588, correct/total - 0/10 (0.00%), bits/total - 235/480 (48.96%). 
TRAIN epoch 26 completed! Avg train loss - 0.696021
	TRAIN data correct/total - 0.0/10.0 (0.00%), bits correct/total - 235.0/480.0 (49.00%)
	VAL   data correct/total - 0.0/10.0 (0.00%), bits correct/total - 248.0/480.0 (52.00%)
TRAIN epoch 27 iter 0: loss - 0.6955558062, correct/total - 0/10 (0.00%), bits/total - 237/480 (49.38%). 
TRAIN epoch 27 completed! Avg train loss - 0.695556
	TRAIN data correct/total - 0.0/10.0 (0.00%), bits correct/total - 237.0/480.0 (49.00%)
	VAL   data correct/total - 0.0/10.0 (0.00%), bits correct/total - 247.0/480.0 (51.00%

TRAIN epoch 50 iter 0: loss - 0.6850115061, correct/total - 0/10 (0.00%), bits/total - 256/480 (53.33%). 
TRAIN epoch 50 completed! Avg train loss - 0.685012
	TRAIN data correct/total - 0.0/10.0 (0.00%), bits correct/total - 256.0/480.0 (53.00%)
	VAL   data correct/total - 0.0/10.0 (0.00%), bits correct/total - 255.0/480.0 (53.00%)
TRAIN epoch 51 iter 0: loss - 0.6845591664, correct/total - 0/10 (0.00%), bits/total - 257/480 (53.54%). 
TRAIN epoch 51 completed! Avg train loss - 0.684559
	TRAIN data correct/total - 0.0/10.0 (0.00%), bits correct/total - 257.0/480.0 (54.00%)
	VAL   data correct/total - 0.0/10.0 (0.00%), bits correct/total - 254.0/480.0 (53.00%)
TRAIN epoch 52 iter 0: loss - 0.6841073632, correct/total - 0/10 (0.00%), bits/total - 259/480 (53.96%). 
TRAIN epoch 52 completed! Avg train loss - 0.684107
	TRAIN data correct/total - 0.0/10.0 (0.00%), bits correct/total - 259.0/480.0 (54.00%)
	VAL   data correct/total - 0.0/10.0 (0.00%), bits correct/total - 255.0/480.0 (53.00%

TRAIN epoch 75 iter 0: loss - 0.6736570597, correct/total - 0/10 (0.00%), bits/total - 289/480 (60.21%). 
TRAIN epoch 75 completed! Avg train loss - 0.673657
	TRAIN data correct/total - 0.0/10.0 (0.00%), bits correct/total - 289.0/480.0 (60.00%)
	VAL   data correct/total - 0.0/10.0 (0.00%), bits correct/total - 271.0/480.0 (56.00%)
TRAIN epoch 76 iter 0: loss - 0.6731914878, correct/total - 0/10 (0.00%), bits/total - 291/480 (60.62%). 
TRAIN epoch 76 completed! Avg train loss - 0.673191
	TRAIN data correct/total - 0.0/10.0 (0.00%), bits correct/total - 291.0/480.0 (61.00%)
	VAL   data correct/total - 0.0/10.0 (0.00%), bits correct/total - 272.0/480.0 (57.00%)
TRAIN epoch 77 iter 0: loss - 0.6727246046, correct/total - 0/10 (0.00%), bits/total - 291/480 (60.62%). 
TRAIN epoch 77 completed! Avg train loss - 0.672725
	TRAIN data correct/total - 0.0/10.0 (0.00%), bits correct/total - 291.0/480.0 (61.00%)
	VAL   data correct/total - 0.0/10.0 (0.00%), bits correct/total - 274.0/480.0 (57.00%

TRAIN epoch 100 iter 0: loss - 0.6615442038, correct/total - 0/10 (0.00%), bits/total - 316/480 (65.83%). 
TRAIN epoch 100 completed! Avg train loss - 0.661544
	TRAIN data correct/total - 0.0/10.0 (0.00%), bits correct/total - 316.0/480.0 (66.00%)
	VAL   data correct/total - 0.0/10.0 (0.00%), bits correct/total - 287.0/480.0 (60.00%)
TRAIN epoch 101 iter 0: loss - 0.6610332131, correct/total - 0/10 (0.00%), bits/total - 319/480 (66.46%). 
TRAIN epoch 101 completed! Avg train loss - 0.661033
	TRAIN data correct/total - 0.0/10.0 (0.00%), bits correct/total - 319.0/480.0 (66.00%)
	VAL   data correct/total - 0.0/10.0 (0.00%), bits correct/total - 287.0/480.0 (60.00%)
TRAIN epoch 102 iter 0: loss - 0.6605195403, correct/total - 0/10 (0.00%), bits/total - 322/480 (67.08%). 
TRAIN epoch 102 completed! Avg train loss - 0.660520
	TRAIN data correct/total - 0.0/10.0 (0.00%), bits correct/total - 322.0/480.0 (67.00%)
	VAL   data correct/total - 0.0/10.0 (0.00%), bits correct/total - 290.0/480.0 (

...

TRAIN epoch 897 iter 0: loss - 0.1458081305, correct/total - 10/10 (100.00%), bits/total - 480/480 (100.00%). 
TRAIN epoch 897 completed! Avg train loss - 0.145808
	TRAIN data correct/total - 10.0/10.0 (100.00%), bits correct/total - 480.0/480.0 (100.00%)
	VAL   data correct/total - 5.0/10.0 (50.00%), bits correct/total - 370.0/480.0 (77.00%)
TRAIN epoch 898 iter 0: loss - 0.1454381794, correct/total - 10/10 (100.00%), bits/total - 480/480 (100.00%). 
TRAIN epoch 898 completed! Avg train loss - 0.145438
	TRAIN data correct/total - 10.0/10.0 (100.00%), bits correct/total - 480.0/480.0 (100.00%)
	VAL   data correct/total - 5.0/10.0 (50.00%), bits correct/total - 370.0/480.0 (77.00%)
TRAIN epoch 899 iter 0: loss - 0.1450691372, correct/total - 10/10 (100.00%), bits/total - 480/480 (100.00%). 
TRAIN epoch 899 completed! Avg train loss - 0.145069
	TRAIN data correct/total - 10.0/10.0 (100.00%), bits correct/total - 480.0/480.0 (100.00%)
	VAL   data correct/total - 5.0/10.0 (50.00%), bits co

TRAIN epoch 921 iter 0: loss - 0.1371654123, correct/total - 10/10 (100.00%), bits/total - 480/480 (100.00%). 
TRAIN epoch 921 completed! Avg train loss - 0.137165
	TRAIN data correct/total - 10.0/10.0 (100.00%), bits correct/total - 480.0/480.0 (100.00%)
	VAL   data correct/total - 5.0/10.0 (50.00%), bits correct/total - 370.0/480.0 (77.00%)
TRAIN epoch 922 iter 0: loss - 0.1368161291, correct/total - 10/10 (100.00%), bits/total - 480/480 (100.00%). 
TRAIN epoch 922 completed! Avg train loss - 0.136816
	TRAIN data correct/total - 10.0/10.0 (100.00%), bits correct/total - 480.0/480.0 (100.00%)
	VAL   data correct/total - 5.0/10.0 (50.00%), bits correct/total - 370.0/480.0 (77.00%)
TRAIN epoch 923 iter 0: loss - 0.1364677399, correct/total - 10/10 (100.00%), bits/total - 480/480 (100.00%). 
TRAIN epoch 923 completed! Avg train loss - 0.136468
	TRAIN data correct/total - 10.0/10.0 (100.00%), bits correct/total - 480.0/480.0 (100.00%)
	VAL   data correct/total - 5.0/10.0 (50.00%), bits co

TRAIN epoch 945 iter 0: loss - 0.1290258616, correct/total - 10/10 (100.00%), bits/total - 480/480 (100.00%). 
TRAIN epoch 945 completed! Avg train loss - 0.129026
	TRAIN data correct/total - 10.0/10.0 (100.00%), bits correct/total - 480.0/480.0 (100.00%)
	VAL   data correct/total - 5.0/10.0 (50.00%), bits correct/total - 370.0/480.0 (77.00%)
TRAIN epoch 946 iter 0: loss - 0.1286974549, correct/total - 10/10 (100.00%), bits/total - 480/480 (100.00%). 
TRAIN epoch 946 completed! Avg train loss - 0.128697
	TRAIN data correct/total - 10.0/10.0 (100.00%), bits correct/total - 480.0/480.0 (100.00%)
	VAL   data correct/total - 5.0/10.0 (50.00%), bits correct/total - 370.0/480.0 (77.00%)
TRAIN epoch 947 iter 0: loss - 0.1283698678, correct/total - 10/10 (100.00%), bits/total - 480/480 (100.00%). 
TRAIN epoch 947 completed! Avg train loss - 0.128370
	TRAIN data correct/total - 10.0/10.0 (100.00%), bits correct/total - 480.0/480.0 (100.00%)
	VAL   data correct/total - 5.0/10.0 (50.00%), bits co

TRAIN epoch 969 iter 0: loss - 0.1213724166, correct/total - 10/10 (100.00%), bits/total - 480/480 (100.00%). 
TRAIN epoch 969 completed! Avg train loss - 0.121372
	TRAIN data correct/total - 10.0/10.0 (100.00%), bits correct/total - 480.0/480.0 (100.00%)
	VAL   data correct/total - 5.0/10.0 (50.00%), bits correct/total - 370.0/480.0 (77.00%)
TRAIN epoch 970 iter 0: loss - 0.1210636869, correct/total - 10/10 (100.00%), bits/total - 480/480 (100.00%). 
TRAIN epoch 970 completed! Avg train loss - 0.121064
	TRAIN data correct/total - 10.0/10.0 (100.00%), bits correct/total - 480.0/480.0 (100.00%)
	VAL   data correct/total - 5.0/10.0 (50.00%), bits correct/total - 370.0/480.0 (77.00%)
TRAIN epoch 971 iter 0: loss - 0.1207557619, correct/total - 10/10 (100.00%), bits/total - 480/480 (100.00%). 
TRAIN epoch 971 completed! Avg train loss - 0.120756
	TRAIN data correct/total - 10.0/10.0 (100.00%), bits correct/total - 480.0/480.0 (100.00%)
	VAL   data correct/total - 5.0/10.0 (50.00%), bits co

TRAIN epoch 993 iter 0: loss - 0.1141813919, correct/total - 10/10 (100.00%), bits/total - 480/480 (100.00%). 
TRAIN epoch 993 completed! Avg train loss - 0.114181
	TRAIN data correct/total - 10.0/10.0 (100.00%), bits correct/total - 480.0/480.0 (100.00%)
	VAL   data correct/total - 5.0/10.0 (50.00%), bits correct/total - 370.0/480.0 (77.00%)
TRAIN epoch 994 iter 0: loss - 0.1138915196, correct/total - 10/10 (100.00%), bits/total - 480/480 (100.00%). 
TRAIN epoch 994 completed! Avg train loss - 0.113892
	TRAIN data correct/total - 10.0/10.0 (100.00%), bits correct/total - 480.0/480.0 (100.00%)
	VAL   data correct/total - 5.0/10.0 (50.00%), bits correct/total - 370.0/480.0 (77.00%)
TRAIN epoch 995 iter 0: loss - 0.1136024520, correct/total - 10/10 (100.00%), bits/total - 480/480 (100.00%). 
TRAIN epoch 995 completed! Avg train loss - 0.113602
	TRAIN data correct/total - 10.0/10.0 (100.00%), bits correct/total - 480.0/480.0 (100.00%)
	VAL   data correct/total - 5.0/10.0 (50.00%), bits co